In [ ]:
import numpy as np
import seaborn as sb
import pandas
import sys
import itertools
import matplotlib.pyplot as plt
import nltk
import csv
import datetime
import collections
import time
import tensorflow as tf
import tflearn
%matplotlib notebook

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


# Generative Adverserial Networks (GANs)

Highlighted by Yann Le Cun as one of *the* big developments in deep learning in the past couple of years, GANs have taken off big.

They were developed by Ian Goodfellow (a lead author on the new Deep Learning book http://www.deeplearningbook.org/) and have since then been extended in several applications.

Here is the classic publication from 2014:

Goodfellow, I., Pouget-Abadie, J., Mirza, M., Xu, B., Warde-Farley, D., Ozair, S., Courville, A. and Bengio, Y., 2014. Generative adversarial nets. In Advances in neural information processing systems (pp. 2672-2680).

In order to solve a certain task, a learning algorithms needs to be able to capture the relevant distributions of the data. In general, we have two approaches:

* discriminative approach: here we try to train a network that maps the input data $x$ to some desired output class label $y$. In probabilistic terms, this directly learns the conditional distribution $P(y|x)$.

* generative approach: we learn the joint probability distribution of the input data and labels simultaneously, i.e. $P(x,y)$. This can be converted to $P(y|x)$ for classification via Bayes' rule, of course. Importantly, however, we can use $P(x,y)$ to create new samples $(x,y)$ that have highlikelihood.

So far, we have used discriminative approaches in our DNNs.

What about combining the two approaches?

The basic idea of GANs is to let the two fight against each other. You have one generator that takes noise samples and tries to turn them into something that can fool a discriminator into thinking it belongs to one of the actual training samples. Both networks are trained together - in competition - and you use gradient updates from the discriminator to update the generator!

![title](images/gan.png)

In [ ]:
# execution mode
training=False

# let's get MNIST - this is NOT one-hot encoded
import tflearn.datasets.mnist as mnist
X, Y, testX, testY = mnist.load_data()

# this is the input dimension for the discriminator
image_dim = 784 # 28*28 pixels

# this is the input dimension for the generator and
# determines the number of noise data points
z_dim = 200 

# how many samples we have in our training data?
total_samples = len(X)


# generator network
def generator(x):
    # this uses the tensorflow scope to define a set of networks
    # whose parameters will exist as named, global variables
    # hence, if reuse is True, the weights are shared across 
    # different calls!!! here, we don't need that
    with tf.variable_scope('Generator', reuse=False):
        # we add a standard hidden layer
        x = tflearn.fully_connected(x, 256, activation='relu')
        # and this guy should give us a picture in the end!
        x = tflearn.fully_connected(x, image_dim, activation='sigmoid')
        return x


# discriminator network - note the reuse parameter!!
def discriminator(x, reuse=False):
    # this uses the tensorflow scope to define a set of networks
    # whose parameters will exist as named, global variables
    # hence, if reuse is True, the weights are shared across 
    # different calls!!!
    with tf.variable_scope('Discriminator', reuse=reuse):
        # we add a standard hidden layer
        x = tflearn.fully_connected(x, 256, activation='relu')
        # and this guy should give tell us whether it was a fake or a real picture!!
        x = tflearn.fully_connected(x, 1, activation='sigmoid')
        return x

# let's set up the two networks

# the generator input consists of noise sampled from a z_dim-dimensional
# noise distribution
gen_input = tflearn.input_data(shape=[None, z_dim], name='input_noise')

# the discriminator input consists of digit pictures
disc_input = tflearn.input_data(shape=[None, 784], name='disc_input')

# we now add the generator layer - this will produce a picture
gen_sample = generator(gen_input)

# this will be used for the proper MNIST digit pictures:
disc_real = discriminator(disc_input)

# here comes the trick: we use the SAME network from the MNIST pictures
# to classify the generated sample from the generator network
# *****note the reuse parameter!!*****
disc_fake = discriminator(gen_sample, reuse=True)

# we also need to define how to train all networks
# the discriminator network loss should weight both the real loss on
# the actual MNIST digit pictures (tf.log(disc_real)) and should be 
# punished by the loss on the fake digit pictures (tf.log(disc_fake))
disc_loss = -tf.reduce_mean(tf.log(disc_real) + tf.log(1. - disc_fake))

# the generator loss is solely driven by the on the fake pictures, since
# of course it tries to make these as well as possible! In other words,
# the generator needs to fool the discriminator!
gen_loss = -tf.reduce_mean(tf.log(disc_fake))

# let's train both networks against each other
# we need to be careful that each network optimization should only 
# update its own variable - for this, we retrieve each network's 
# variables (with get_layer_variables_by_scope) and set
# 'placeholder=None' because we do not need to feed any target.

# get the optimization variables for the generator network:
gen_vars = tflearn.get_layer_variables_by_scope('Generator')
gen_model = tflearn.regression(gen_sample, placeholder=None, optimizer='adam',
                               loss=gen_loss, trainable_vars=gen_vars,
                               batch_size=64, name='target_gen', op_name='GEN')

# get the optimization variables for the discriminator network:
disc_vars = tflearn.get_layer_variables_by_scope('Discriminator')
disc_model = tflearn.regression(disc_real, placeholder=None, optimizer='adam',
                                loss=disc_loss, trainable_vars=disc_vars,
                                batch_size=64, name='target_disc', op_name='DISC')

# and define the GAN model, that outputs the generated images
# importantly, the GAN model uses the fake loss, which means that it also 
# will update the discriminator model in each step
gan = tflearn.DNN(gen_model)

if (training):
    # Training
    # Generate noise to feed to the generator - this is simply uniformly
    # distributed noise between -1 and 1
    z = np.random.uniform(-1., 1., size=[total_samples, z_dim])
    # Start training, feed both noise and real images as inputs to both
    # networks - we train for 100 epochs
    gan.fit(X_inputs={gen_input: z, disc_input: X},
            Y_targets=None,
            n_epoch=100)
    gan.save('ganTrained.tflearn')
else:
    # Sampling
    gan.load('ganTrained.tflearn')
    # Generate 20 images from noise, using the generator network.
    f, a = plt.subplots(2, 10, figsize=(10, 4))
    for i in range(10):
        for j in range(2):
            # Same as before - now we just put in one noise image
            z = np.random.uniform(-1., 1., size=[1, z_dim])
            # we use the gan network to "predict", that is to create an image
            # this is extended three times, since matplotlib requires 3 channels
            temp = [[ii, ii, ii] for ii in list(gan.predict([z])[0])]
            a[j][i].imshow(np.reshape(temp, (28, 28, 3)))
    f.show()
    plt.draw()

In [ ]:
# execution mode
training=True

# let's get MNIST - this is NOT one-hot encoded
import tflearn.datasets.mnist as mnist
X, Y, testX, testY = mnist.load_data()

# this is the input dimension for the discriminator
image_dim = 784 # 28*28 pixels

# this is the input dimension for the generator and
# determines the number of noise data points
z_dim = 200 

# how many samples we have in our training data?
total_samples = len(X)


# generator network
def generator(x):
    # this uses the tensorflow scope to define a set of networks
    # whose parameters will exist as named, global variables
    # hence, if reuse is True, the weights are shared across 
    # different calls!!! here, we don't need that
    with tf.variable_scope('Generator', reuse=False):
        # we add a standard hidden layer
        x = tflearn.fully_connected(x, 256, activation='relu')
        # and this guy should give us a picture in the end!
        x = tflearn.fully_connected(x, image_dim, activation='sigmoid')
        return x


# discriminator network - note the reuse parameter!!
def discriminator(x, reuse=False):
    # this uses the tensorflow scope to define a set of networks
    # whose parameters will exist as named, global variables
    # hence, if reuse is True, the weights are shared across 
    # different calls!!!
    with tf.variable_scope('Discriminator', reuse=reuse):
        # we add a standard hidden layer
        x = tflearn.fully_connected(x, 256, activation='relu')
        # and this guy should give tell us whether it was a fake or a real picture!!
        x = tflearn.fully_connected(x, 1, activation='sigmoid')
        return x

# let's set up the two networks

# the generator input consists of noise sampled from a z_dim-dimensional
# noise distribution
gen_input = tflearn.input_data(shape=[None, z_dim], name='input_noise')

# the discriminator input consists of digit pictures
disc_input = tflearn.input_data(shape=[None, 784], name='disc_input')

# we now add the generator layer - this will produce a picture
gen_sample = generator(gen_input)

# this will be used for the proper MNIST digit pictures:
disc_real = discriminator(disc_input)

# here comes the trick: we use the SAME network from the MNIST pictures
# to classify the generated sample from the generator network
# *****note the reuse parameter!!*****
disc_fake = discriminator(gen_sample, reuse=True)

# we also need to define how to train all networks
# the discriminator network loss should weight both the real loss on
# the actual MNIST digit pictures (tf.log(disc_real)) and should be 
# punished by the loss on the fake digit pictures (tf.log(disc_fake))
disc_loss = -tf.reduce_mean(tf.log(disc_real) + tf.log(1. - disc_fake))

# the generator loss is solely driven by the on the fake pictures, since
# of course it tries to make these as well as possible! In other words,
# the generator needs to fool the discriminator!
gen_loss = -tf.reduce_mean(tf.log(disc_fake))

# let's train both networks against each other
# we need to be careful that each network optimization should only 
# update its own variable - for this, we retrieve each network's 
# variables (with get_layer_variables_by_scope) and set
# 'placeholder=None' because we do not need to feed any target.

# get the optimization variables for the generator network:
gen_vars = tflearn.get_layer_variables_by_scope('Generator')
gen_model = tflearn.regression(gen_sample, placeholder=None, optimizer='adam',
                               loss=gen_loss, trainable_vars=gen_vars,
                               batch_size=64, name='target_gen', op_name='GEN')

# get the optimization variables for the discriminator network:
disc_vars = tflearn.get_layer_variables_by_scope('Discriminator')
disc_model = tflearn.regression(disc_real, placeholder=None, optimizer='adam',
                                loss=disc_loss, trainable_vars=disc_vars,
                                batch_size=64, name='target_disc', op_name='DISC')

# and define the GAN model, that outputs the generated images
# importantly, the GAN model uses the fake loss, which means that it also 
# will update the discriminator model in each step
gan = tflearn.DNN(gen_model)

disc = tflearn.DNN(disc_model)

# Training
# Generate noise to feed to the generator - this is simply uniformly
# distributed noise between -1 and 1
z = np.random.uniform(-1., 1., size=[total_samples, z_dim])
# Start training, feed both noise and real images as inputs to both
# networks - we train for 100 epochs
gan.fit(X_inputs={gen_input: z, disc_input: X},
        Y_targets=None,
        n_epoch=100)
gan.save('ganTrained1.tflearn')
disc.save('discTrained1.tflearn')


Training Step: 8927  | time: 4.002s
| GEN | epoch: 011 | loss: 0.00000 -- iter: 20928/55000
| DISC | epoch: 011 | loss: 0.00000 -- iter: 20928/55000
